In [2]:
import yfinance as yf
import pandas as pd

In [7]:
ticker_symbol = 'BTC-USD'
df = yf.download(ticker_symbol, start='2024-01-01', end='2024-8-17', interval='60m')
df.index.name = df.index.name.lower()
df.columns = df.columns.str.lower()
df['year'] = df.index.year
df['month'] = df.index.month
df['hour'] = df.index.hour
df['day'] = df.index.day
df['day_name'] = df.index.day_name()
df['candle_body'] = df['open'] - df['close']
df['high_shadow'] = df['high'] - df['open']
df['low_shadow'] = df['close'] - df['low']
df['trend'] = df['candle_body'].apply(lambda x: "UP" if x > 0 else "ZERO" if x == 0 else "DOWN")

[*********************100%%**********************]  1 of 1 completed


In [8]:
df.to_csv('trend_2024.csv', index=False)


In [6]:
january = df[df['month']==1]
january_tuesday = january[january['day_name']=='Tuesday']

In [8]:
january_tuesday

,open,high,low,close,adj close,volume,year,month,hour,day,day_name,candle_body,high_shadow,low_shadow,trend
datetime,,,,,,,,,,,,,,,
2024-01-02 00:00:00,44187.140625,45069.960938,44176.949219,45069.960938,45069.960938,2670448640,2024,1,0,2,Tuesday,-882.820312,882.820312,-893.011719,DOWN
2024-01-02 01:00:00,45121.183594,45381.257812,44754.218750,44883.664062,44883.664062,2205095936,2024,1,1,2,Tuesday,237.519531,260.074219,-129.445312,UP
2024-01-02 02:00:00,44882.656250,45398.496094,44882.656250,45398.496094,45398.496094,1181814784,2024,1,2,2,Tuesday,-515.839844,515.839844,-515.839844,DOWN
2024-01-02 03:00:00,45412.664062,45474.234375,45224.281250,45421.730469,45421.730469,1125044224,2024,1,3,2,Tuesday,-9.066406,61.570312,-197.449219,DOWN
2024-01-02 04:00:00,45426.203125,45460.750000,45215.113281,45216.746094,45216.746094,678227968,2024,1,4,2,Tuesday,209.457031,34.546875,-1.632812,UP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-30 19:00:00,43372.320312,43554.824219,43344.351562,43534.574219,43534.574219,0,2024,1,19,30,Tuesday,-162.253906,182.503906,-190.222656,DOWN
2024-01-30 20:00:00,43545.546875,43731.906250,43511.488281,43641.152344,43641.152344,759971840,2024,1,20,30,Tuesday,-95.605469,186.359375,-129.664062,DOWN
2024-01-30 21:00:00,43571.070312,43683.093750,43500.671875,43545.640625,43545.640625,159399936,2024,1,21,30,Tuesday,25.429688,112.023438,-44.968750,UP


In [9]:
january_dfs = [group.assign(volume=group['volume'] / max(group['volume'])) for _, group in january_tuesday[['day','hour','volume', 'trend']].groupby('day')]
total_days = len(january_dfs)

most_repeating_january, least_repeating_january = [], []
for january_df in january_dfs:
    january_up_count = 0
    january_down_count = 0
    january_up_volume = 0
    january_down_volume = 0
    january_up_hours = []
    sorted_hours = []
    january_sorted_up_hours = []
    january_down_hours = []
    january_sorted_down_hours = []

    print("~~~~~~~~~~~~~~~~",january_df['day'].iloc[0]," January ~~~~~~~~~~~~~~~~~~")
    january_hour_volume = {hour: volume for hour, volume in zip(january_df['hour'], january_df['volume'])}
    january_hour_trend = {hour: trend for hour, trend in zip(january_df['hour'], january_df['trend'])}

    print(january_hour_trend)
    print(january_hour_volume)
    print("-----------------")


    for hour, trend in january_hour_trend.items():
        if trend == "UP":
            january_up_count+=1
            january_up_volume+=january_hour_volume.get(hour)
            january_up_hours.append(hour)
        else:
            january_down_count+=1
            january_down_volume+=january_hour_volume.get(hour)
            january_down_hours.append(hour)

    for hour in dict(sorted(january_hour_volume.items(), key=lambda item: item[1])).keys():
        sorted_hours.append(hour)
        if january_hour_trend.get(hour) == "UP":
            january_sorted_up_hours.append(hour)
        else:
            january_sorted_down_hours.append(hour)

    print("UP:::", january_up_count, january_up_volume)
    print("DOWN:::", january_down_count, january_down_volume)
    print("-----------------")

    print("UP Hours:::", january_up_hours)
    print("DOWN Hours:::", january_down_hours)
    print("-----------------")

    print("SORTED UP Hours:::", january_sorted_up_hours)
    print("SORTED DOWN Hours:::", january_sorted_down_hours)
    print("-----------------")
    print("SORTED HOURS ASCENDING:::", sorted_hours)
    print("SORTED HOURS DESCENDING:::", list(reversed(sorted_hours)))


NameError: name 'january_tuesday' is not defined

In [12]:
february_dfs = [group.assign(volume=group['volume'] / tuesday_max_volume) for _, group in february_tuesday[['day','hour','volume']].groupby('day')]

total_days = len(february_dfs)

most_repeating_february, least_repeating_february = [], []
for february_df in february_dfs:
    print("~~~~~~~~~~~~~~~~",february_df['day'].iloc[0]," february ~~~~~~~~~~~~~~~~~~")
    result = {hour: volume for hour, volume in zip(february_df['hour'], february_df['volume'])}
    print(result)
    print("==========")
    hours = list(result.keys())
    print(hours)
    most_repeating_february.extend(hours[:len(hours)//2])
    least_repeating_february.extend(hours[len(hours)//2:])

from collections import Counter
most_repeating_february_counts = Counter(most_repeating_february)
least_repeating_february_counts = Counter(least_repeating_february)

print("*"*20)
print(most_repeating_february_counts)
print(least_repeating_february_counts)
print("*"*20)

make_trades = [key for key, value in most_repeating_february_counts.items() if value >= total_days]
avoid_trades = [key for key, value in least_repeating_february_counts.items() if value >= total_days]

print("Make Trades",make_trades)
print('Avoid Trades',avoid_trades)

NameError: name 'february_tuesday' is not defined

In [11]:
march_dfs = [group.assign(volume=group['volume'] / tuesday_max_volume) for _, group in march_tuesday[['day','hour','volume']].groupby('day')]

total_days = len(march_dfs)

most_repeating_march, least_repeating_march = [], []
for march_df in march_dfs:
    print("~~~~~~~~~~~~~~~~",march_df['day'].iloc[0]," march ~~~~~~~~~~~~~~~~~~")
    result = {hour: volume for hour, volume in zip(march_df['hour'], march_df['volume'])}
    print(result)
    print("==========")
    hours = list(result.keys())
    print(hours)
    most_repeating_march.extend(hours[:len(hours)//2])
    least_repeating_march.extend(hours[len(hours)//2:])

from collections import Counter
most_repeating_march_counts = Counter(most_repeating_march)
least_repeating_march_counts = Counter(least_repeating_march)

print("*"*20)
print(most_repeating_march_counts)
print(least_repeating_march_counts)
print("*"*20)

make_trades = [key for key, value in most_repeating_march_counts.items() if value >= total_days]
avoid_trades = [key for key, value in least_repeating_march_counts.items() if value >= total_days]

print("Make Trades",make_trades)
print('Avoid Trades',avoid_trades)

~~~~~~~~~~~~~~~~ 6  march ~~~~~~~~~~~~~~~~~~
{12: 0.1759536741267714, 13: 0.10696466778853744, 8: 0.08891262507683471, 11: 0.07509816369236556, 6: 0.03993120984021739, 10: 0.032753252249619606, 9: 0.03217350705535794, 5: 0.0024177322759533384, 0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 7: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.0, 18: 0.0, 19: 0.0, 20: 0.0, 21: 0.0, 22: 0.0, 23: 0.0}
[12, 13, 8, 11, 6, 10, 9, 5, 0, 1, 2, 3, 4, 7, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
~~~~~~~~~~~~~~~~ 13  march ~~~~~~~~~~~~~~~~~~
{7: 0.1053916974845239, 8: 0.06458515973222893, 1: 0.04926692127087133, 9: 0.03787523050413648, 10: 0.02814787197237645, 13: 0.014629665084627347, 6: 0.014100303308175213, 2: 0.007958900566645506, 0: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 11: 0.0, 12: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.0, 18: 0.0, 19: 0.0, 20: 0.0, 21: 0.0, 22: 0.0, 23: 0.0}
[7, 8, 1, 9, 10, 13, 6, 2, 0, 3, 4, 5, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
~~~~~~~~~~~~~~~~ 20  march ~~~~~~~~~~~~~~~~~~
{20: 0.17344693

In [12]:
april_dfs = [group.assign(volume=group['volume'] / tuesday_max_volume) for _, group in april_tuesday[['day','hour','volume']].groupby('day')]

total_days = len(april_dfs)

most_repeating_april, least_repeating_april = [], []
for april_df in april_dfs:
    print("~~~~~~~~~~~~~~~~",april_df['day'].iloc[0]," april ~~~~~~~~~~~~~~~~~~")
    result = {hour: volume for hour, volume in zip(april_df['hour'], april_df['volume'])}
    print(result)
    print("==========")
    hours = list(result.keys())
    print(hours)
    most_repeating_april.extend(hours[:len(hours)//2])
    least_repeating_april.extend(hours[len(hours)//2:])

from collections import Counter
most_repeating_april_counts = Counter(most_repeating_april)
least_repeating_april_counts = Counter(least_repeating_april)

print("*"*20)
print(most_repeating_april_counts)
print(least_repeating_april_counts)
print("*"*20)

make_trades = [key for key, value in most_repeating_april_counts.items() if value >= total_days]
avoid_trades = [key for key, value in least_repeating_april_counts.items() if value >= total_days]

print("Make Trades",make_trades)
print('Avoid Trades',avoid_trades)

~~~~~~~~~~~~~~~~ 3  april ~~~~~~~~~~~~~~~~~~
{0: 0.10064994608974294, 1: 0.07842951527793172, 22: 0.02929392678281724, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.0, 18: 0.0, 19: 0.0, 20: 0.0, 21: 0.0, 23: 0.0}
[0, 1, 22, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23]
~~~~~~~~~~~~~~~~ 10  april ~~~~~~~~~~~~~~~~~~
{12: 0.1918232751236913, 22: 0.15726142611943558, 13: 0.06748758048750995, 19: 0.04468538914472469, 2: 0.03993221750857358, 20: 0.03966787250979958, 18: 0.03786515382057457, 3: 0.03262124769495864, 16: 0.025196747246546216, 21: 0.019440273817081324, 10: 0.01565160874253066, 17: 0.012963989291844268, 23: 0.004927162372320022, 0: 0.0, 1: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 11: 0.0, 14: 0.0, 15: 0.0}
[12, 22, 13, 19, 2, 20, 18, 3, 16, 21, 10, 17, 23, 0, 1, 4, 5, 6, 7, 8, 9, 11, 14, 15]
~~~~~~~~~~~~~~~~ 17  april ~~~~~~~~~~~~~~~~~~
{15: 0.205287571

In [13]:
march_dfs = [group.assign(volume=group['volume'] / tuesday_max_volume) for _, group in march_tuesday[['day','hour','volume']].groupby('day')]

total_days = len(march_dfs)

most_repeating_march, least_repeating_march = [], []
for march_df in march_dfs:
    print("~~~~~~~~~~~~~~~~",march_df['day'].iloc[0]," march ~~~~~~~~~~~~~~~~~~")
    result = {hour: volume for hour, volume in zip(march_df['hour'], march_df['volume'])}
    print(result)
    print("==========")
    hours = list(result.keys())
    print(hours)
    most_repeating_march.extend(hours[:len(hours)//2])
    least_repeating_march.extend(hours[len(hours)//2:])

from collections import Counter
most_repeating_march_counts = Counter(most_repeating_march)
least_repeating_march_counts = Counter(least_repeating_march)

print("*"*20)
print(most_repeating_march_counts)
print(least_repeating_march_counts)
print("*"*20)

make_trades = [key for key, value in most_repeating_march_counts.items() if value >= total_days]
avoid_trades = [key for key, value in least_repeating_march_counts.items() if value >= total_days]

print("Make Trades",make_trades)
print('Avoid Trades',avoid_trades)

~~~~~~~~~~~~~~~~ 6  march ~~~~~~~~~~~~~~~~~~
{12: 0.1759536741267714, 13: 0.10696466778853744, 8: 0.08891262507683471, 11: 0.07509816369236556, 6: 0.03993120984021739, 10: 0.032753252249619606, 9: 0.03217350705535794, 5: 0.0024177322759533384, 0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 7: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.0, 18: 0.0, 19: 0.0, 20: 0.0, 21: 0.0, 22: 0.0, 23: 0.0}
[12, 13, 8, 11, 6, 10, 9, 5, 0, 1, 2, 3, 4, 7, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
~~~~~~~~~~~~~~~~ 13  march ~~~~~~~~~~~~~~~~~~
{7: 0.1053916974845239, 8: 0.06458515973222893, 1: 0.04926692127087133, 9: 0.03787523050413648, 10: 0.02814787197237645, 13: 0.014629665084627347, 6: 0.014100303308175213, 2: 0.007958900566645506, 0: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 11: 0.0, 12: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.0, 18: 0.0, 19: 0.0, 20: 0.0, 21: 0.0, 22: 0.0, 23: 0.0}
[7, 8, 1, 9, 10, 13, 6, 2, 0, 3, 4, 5, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
~~~~~~~~~~~~~~~~ 20  march ~~~~~~~~~~~~~~~~~~
{20: 0.17344693

In [14]:
april_dfs = [group.assign(volume=group['volume'] / tuesday_max_volume) for _, group in april_tuesday[['day','hour','volume']].groupby('day')]

total_days = len(april_dfs)

most_repeating_april, least_repeating_april = [], []
for april_df in april_dfs:
    print("~~~~~~~~~~~~~~~~",april_df['day'].iloc[0]," april ~~~~~~~~~~~~~~~~~~")
    result = {hour: volume for hour, volume in zip(april_df['hour'], april_df['volume'])}
    print(result)
    print("==========")
    hours = list(result.keys())
    print(hours)
    most_repeating_april.extend(hours[:len(hours)//2])
    least_repeating_april.extend(hours[len(hours)//2:])

from collections import Counter
most_repeating_april_counts = Counter(most_repeating_april)
least_repeating_april_counts = Counter(least_repeating_april)

print("*"*20)
print(most_repeating_april_counts)
print(least_repeating_april_counts)
print("*"*20)

make_trades = [key for key, value in most_repeating_april_counts.items() if value >= total_days]
avoid_trades = [key for key, value in least_repeating_april_counts.items() if value >= total_days]

print("Make Trades",make_trades)
print('Avoid Trades',avoid_trades)

~~~~~~~~~~~~~~~~ 3  april ~~~~~~~~~~~~~~~~~~
{0: 0.10064994608974294, 1: 0.07842951527793172, 22: 0.02929392678281724, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.0, 18: 0.0, 19: 0.0, 20: 0.0, 21: 0.0, 23: 0.0}
[0, 1, 22, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23]
~~~~~~~~~~~~~~~~ 10  april ~~~~~~~~~~~~~~~~~~
{12: 0.1918232751236913, 22: 0.15726142611943558, 13: 0.06748758048750995, 19: 0.04468538914472469, 2: 0.03993221750857358, 20: 0.03966787250979958, 18: 0.03786515382057457, 3: 0.03262124769495864, 16: 0.025196747246546216, 21: 0.019440273817081324, 10: 0.01565160874253066, 17: 0.012963989291844268, 23: 0.004927162372320022, 0: 0.0, 1: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 11: 0.0, 14: 0.0, 15: 0.0}
[12, 22, 13, 19, 2, 20, 18, 3, 16, 21, 10, 17, 23, 0, 1, 4, 5, 6, 7, 8, 9, 11, 14, 15]
~~~~~~~~~~~~~~~~ 17  april ~~~~~~~~~~~~~~~~~~
{15: 0.205287571

In [15]:
may_dfs = [group.assign(volume=group['volume'] / tuesday_max_volume) for _, group in may_tuesday[['day','hour','volume']].groupby('day')]

total_days = len(may_dfs)

most_repeating_may, least_repeating_may = [], []
for may_df in may_dfs:
    print("~~~~~~~~~~~~~~~~",may_df['day'].iloc[0]," may ~~~~~~~~~~~~~~~~~~")
    result = {hour: volume for hour, volume in zip(may_df['hour'], may_df['volume'])}
    print(result)
    print("==========")
    hours = list(result.keys())
    print(hours)
    most_repeating_may.extend(hours[:len(hours)//2])
    least_repeating_may.extend(hours[len(hours)//2:])

from collections import Counter
most_repeating_may_counts = Counter(most_repeating_may)
least_repeating_may_counts = Counter(least_repeating_may)

print("*"*20)
print(most_repeating_may_counts)
print(least_repeating_may_counts)
print("*"*20)

make_trades = [key for key, value in most_repeating_may_counts.items() if value >= total_days]
avoid_trades = [key for key, value in least_repeating_may_counts.items() if value >= total_days]

print("Make Trades",make_trades)
print('Avoid Trades',avoid_trades)

~~~~~~~~~~~~~~~~ 1  may ~~~~~~~~~~~~~~~~~~
{7: 0.3921791499981526, 18: 0.2669182478662623, 8: 0.16599723898870403, 19: 0.06209621889243812, 10: 0.042304940597950404, 21: 0.03426173177883695, 6: 0.03402459382568009, 23: 0.03106238474793176, 13: 0.02850122767594729, 1: 0.02070422582519641, 9: 0.020452644625600824, 3: 0.013293160954866535, 4: 0.01210847885743844, 15: 0.008970263706808815, 5: 8.1957026303503e-05, 0: 0.0, 2: 0.0, 11: 0.0, 12: 0.0, 14: 0.0, 16: 0.0, 17: 0.0, 20: 0.0, 22: 0.0}
[7, 18, 8, 19, 10, 21, 6, 23, 13, 1, 9, 3, 4, 15, 5, 0, 2, 11, 12, 14, 16, 17, 20, 22]
~~~~~~~~~~~~~~~~ 8  may ~~~~~~~~~~~~~~~~~~
{20: 0.061804666848046974, 13: 0.04837866161488931, 21: 0.024123580447203218, 14: 0.02119966276699013, 19: 0.020001545091479493, 0: 0.01916568419001938, 6: 0.018768159023502186, 16: 0.015741123281505592, 17: 0.013715038106658337, 7: 0.01258712132662898, 9: 0.00864025232015639, 2: 0.005219722085067362, 4: 0.0023159577719780865, 1: 0.0, 3: 0.0, 5: 0.0, 8: 0.0, 10: 0.0, 11: 0.0,

In [16]:
june_dfs = [group.assign(volume=group['volume'] / tuesday_max_volume) for _, group in june_tuesday[['day','hour','volume']].groupby('day')]

total_days = len(june_dfs)

most_repeating_june, least_repeating_june = [], []
for june_df in june_dfs:
    print("~~~~~~~~~~~~~~~~",june_df['day'].iloc[0]," june ~~~~~~~~~~~~~~~~~~")
    result = {hour: volume for hour, volume in zip(june_df['hour'], june_df['volume'])}
    print(result)
    print("==========")
    hours = list(result.keys())
    print(hours)
    most_repeating_june.extend(hours[:len(hours)//2])
    least_repeating_june.extend(hours[len(hours)//2:])

from collections import Counter
most_repeating_june_counts = Counter(most_repeating_june)
least_repeating_june_counts = Counter(least_repeating_june)

print("*"*20)
print(most_repeating_june_counts)
print(least_repeating_june_counts)
print("*"*20)

make_trades = [key for key, value in most_repeating_june_counts.items() if value >= total_days-1]
avoid_trades = [key for key, value in least_repeating_june_counts.items() if value >= total_days-1]

print("Make Trades",make_trades)
print('Avoid Trades',avoid_trades)

~~~~~~~~~~~~~~~~ 5  june ~~~~~~~~~~~~~~~~~~
{1: 0.08458032292411921, 2: 0.0725655572238065, 4: 0.06259434294984835, 7: 0.06093504905665447, 12: 0.038520474141550534, 23: 0.034279701864522345, 3: 0.03390770429636198, 5: 0.032352536133307803, 8: 0.020431819479572882, 0: 0.015651272853078595, 10: 0.009496602478192378, 6: 0.008223581454871573, 11: 0.007866530967328034, 9: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.0, 18: 0.0, 19: 0.0, 20: 0.0, 21: 0.0, 22: 0.0}
[1, 2, 4, 7, 12, 23, 3, 5, 8, 0, 10, 6, 11, 9, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
~~~~~~~~~~~~~~~~ 12  june ~~~~~~~~~~~~~~~~~~
{19: 0.18895444331361663, 12: 0.17132545336678792, 18: 0.16025101018752708, 20: 0.09022141832680028, 21: 0.04415384408683414, 14: 0.04155993779327348, 13: 0.029711101482280153, 0: 0.004363203982305343, 22: 0.004051498570790382, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 15: 0.0, 16: 0.0, 17: 0.0, 23: 0.0}
[19, 12, 18, 20, 21, 14, 13, 0, 22, 1, 2, 3, 4, 

In [17]:
july_dfs = [group.assign(volume=group['volume'] / tuesday_max_volume) for _, group in july_tuesday[['day','hour','volume']].groupby('day')]

total_days = len(july_dfs)

most_repeating_july, least_repeating_july = [], []
for july_df in july_dfs:
    print("~~~~~~~~~~~~~~~~",july_df['day'].iloc[0]," july ~~~~~~~~~~~~~~~~~~")
    result = {hour: volume for hour, volume in zip(july_df['hour'], july_df['volume'])}
    print(result)
    print("==========")
    hours = list(result.keys())
    print(hours)
    most_repeating_july.extend(hours[:len(hours)//2])
    least_repeating_july.extend(hours[len(hours)//2:])

from collections import Counter
most_repeating_july_counts = Counter(most_repeating_july)
least_repeating_july_counts = Counter(least_repeating_july)

print("*"*20)
print(most_repeating_july_counts)
print(least_repeating_july_counts)
print("*"*20)

make_trades = [key for key, value in most_repeating_july_counts.items() if value >= total_days-1]
avoid_trades = [key for key, value in least_repeating_july_counts.items() if value >= total_days-1]

print("Make Trades",make_trades)
print('Avoid Trades',avoid_trades)

~~~~~~~~~~~~~~~~ 3  july ~~~~~~~~~~~~~~~~~~
{10: 0.13197516433391443, 2: 0.09789498080391781, 11: 0.07909726350863404, 12: 0.07758592891907415, 3: 0.0737750951406873, 13: 0.06520873849998489, 21: 0.0619872227652435, 19: 0.058549394223373206, 20: 0.04239092829767867, 8: 0.041760295851429374, 16: 0.036264304692039756, 9: 0.027104263444815042, 1: 0.02425592089131625, 5: 0.02109520114739837, 15: 0.019044428097824445, 22: 0.01857871737253835, 23: 0.01793767235327509, 17: 0.017528559000661702, 4: 0.009814689789296546, 6: 0.007156124776213652, 0: 0.0, 7: 0.0, 14: 0.0, 18: 0.0}
[10, 2, 11, 12, 3, 13, 21, 19, 20, 8, 16, 9, 1, 5, 15, 22, 23, 17, 4, 6, 0, 7, 14, 18]
~~~~~~~~~~~~~~~~ 10  july ~~~~~~~~~~~~~~~~~~
{4: 0.1476813551124054, 13: 0.04842921297742487, 12: 0.04268248034206982, 23: 0.01872029477658313, 5: 0.018372985083149433, 11: 0.016197429102133905, 10: 0.008996127194617707, 17: 0.0022390390874555367, 9: 0.0016329265712068843, 21: 0.0011311077298239603, 20: 7.33918452758828e-05, 0: 0.0, 1

In [18]:
august_dfs = [group.assign(volume=group['volume'] / tuesday_max_volume) for _, group in august_tuesday[['day','hour','volume']].groupby('day')]

total_days = len(august_dfs)

most_repeating_august, least_repeating_august = [], []
for august_df in august_dfs:
    print("~~~~~~~~~~~~~~~~",august_df['day'].iloc[0]," august ~~~~~~~~~~~~~~~~~~")
    result = {hour: volume for hour, volume in zip(august_df['hour'], august_df['volume'])}
    print(result)
    print("==========")
    hours = list(result.keys())
    print(hours)
    most_repeating_august.extend(hours[:len(hours)//2])
    least_repeating_august.extend(hours[len(hours)//2:])

from collections import Counter
most_repeating_august_counts = Counter(most_repeating_august)
least_repeating_august_counts = Counter(least_repeating_august)

print("*"*20)
print(most_repeating_august_counts)
print(least_repeating_august_counts)
print("*"*20)

make_trades = [key for key, value in most_repeating_august_counts.items() if value >= total_days]
avoid_trades = [key for key, value in least_repeating_august_counts.items() if value >= total_days]

print("Make Trades",make_trades)
print('Avoid Trades',avoid_trades)

NameError: name 'august_tuesday' is not defined